In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [3]:
machinedf = pd.read_csv('ML_Dataset.csv', index_col = 0)

In [4]:
machinedf.head(5)

,is_churn,bd,num_25|201609,num_25|201610,num_25|201611,num_25|201612,num_25|201701,num_25|201702,num_50|201609,num_50|201610,...,city_21,city_22,registered_via_3,registered_via_4,registered_via_7,registered_via_9,registered_via_13,gender_female,gender_male,gender_missing
0,1,36.0,0.0,0.0,3.0,0.0,35.0,13.0,0.0,0.0,...,0,0,0,0,0,1,0,1,0,0
1,1,38.0,11.0,1.0,0.0,25.0,0.0,0.0,5.0,0.0,...,0,0,0,0,0,1,0,0,1,0
2,1,27.0,133.0,91.0,17.0,128.0,22.0,0.0,74.0,52.0,...,0,0,0,0,0,1,0,1,0,0
3,1,27.0,43.0,44.0,14.0,26.0,17.0,36.0,34.0,16.0,...,0,0,0,0,0,1,0,0,1,0
4,1,23.0,52.0,56.0,32.0,55.0,44.0,12.0,13.0,12.0,...,0,0,0,0,0,1,0,1,0,0


In [5]:
from sklearn.model_selection import train_test_split
machinedf_train, machinedf_test = train_test_split(machinedf, test_size = 0.2, stratify = machinedf.is_churn)

In [6]:
print(machinedf_train.shape)
print(machinedf_test.shape)

(651326, 103)
(162832, 103)


In [7]:
machinedf_train[machinedf_train.is_churn == 1].shape[0]

36474

In [8]:
difference = machinedf_train[machinedf_train.is_churn == 0].shape[0] - machinedf_train[machinedf_train.is_churn == 1].shape[0]
difference

578378

In [9]:
machinedf_train = machinedf_train.append(machinedf_train[machinedf_train.is_churn == 1].sample(difference, replace = True))

In [10]:
machinedf_train[machinedf_train.is_churn == 1].shape[0]

614852

## Neural Network Architecture

In [11]:
x = tf.placeholder(tf.float32, [None, 102])
y_true = tf.placeholder(tf.float32, [None,1])
y_true_class = tf.cast(y_true, tf.int64)

In [12]:
W1 = tf.Variable(tf.zeros([102,300]))
b1 = tf.Variable(tf.zeros([300]))

In [13]:
W2 = tf.Variable(tf.zeros([300,400]))
b2 = tf.Variable(tf.zeros([400]))

In [14]:
W3 = tf.Variable(tf.zeros([400,1]))
b3 = tf.Variable(tf.zeros([1]))

In [15]:
logits = tf.matmul((tf.matmul((tf.matmul(x,W1) + b1),W2) + b2), W3) + b3
y_pred = tf.nn.sigmoid(logits)
y_pred_class = tf.cast(y_pred, tf.int64)

In [16]:
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits = logits, labels = y_true)
cost = tf.reduce_mean(cross_entropy)

In [17]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5).minimize(cost)

In [18]:
correct_prediction = tf.equal(y_pred_class, y_true_class)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Start Session

In [19]:
sess = tf.Session()

In [20]:
sess.run(tf.global_variables_initializer())

In [21]:
batch_size = 100000

In [22]:
for _ in range(1000):
    batch_data = machinedf_train.sample(batch_size)
    feed_dict = {x: batch_data.drop('is_churn', axis = 1).values,
                y_true: np.reshape(batch_data.is_churn.values, [-1,1])}
    sess.run(optimizer, feed_dict = feed_dict)

In [23]:
test_feed_dict = {x:machinedf_test.drop('is_churn', axis = 1).values,
                 y_true_class: np.reshape(machinedf_test.is_churn.values, [-1,1])}

In [24]:
acc = sess.run(accuracy, feed_dict = test_feed_dict)

In [25]:
acc

0.9439975